# Earnings Call Analysis; Running all models

This notebook orchestrates the complete workflow for analyzing earnings call transcripts and predicting stock returns. It integrates multiple components of the project:

1. **Data Preparation**: Loads and prepares earnings call transcript data along with financial metrics
2. **Baseline Models**: Runs traditional machine learning approaches to establish performance benchmarks:
   - Random baseline model
   - Finance-only logistic regression
   - TF-IDF with logistic regression (transcript-only)
   - TF-IDF + finance features with logistic regression
   - SVM with mean-pooled BERT embeddings (transcript-only and with finance features)
3. **BERT-based Models**: Implements and evaluates advanced deep learning architectures:
    - **AttnMLPPoolClassifier**: Uses attention-based pooling on BERT embeddings for transcript-only analysis
    - **AttnPoolTwoTower**: Combines transcript embeddings with financial features in a two-tower architecture
    - **MeanPoolClassifier**: Uses simple mean pooling of BERT embeddings as an alternative approach

The notebook compares model performance across different architectures and input configurations (transcript-only vs. transcript + financial data) using metrics such as AUC, confidence intervals, and standard errors for 1-day and 5-day return predictions.

In [1]:
# Necessary Imports
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from config import * 
from data_cleaning_util import prepare_earnings_data
from baseline_models import call_baseline_model
from finbert_models_utils import call_model, call_model_fin, meanpooling_withsvm, meanpooling_withsvm_fin

In [ ]:
# Baseline Models
baselinemodels = ["random", "finance_only", "tfidf", "finance_tfidf"]
returndays = [1,5]
raw_data = prepare_earnings_data()
baseline_results = {}
for model in baselinemodels:
    for days in returndays:
        result_df = call_baseline_model(raw_data, model, days)
        baseline_results[(model, days)] = result_df

# Add model and return period information to baseline results
baseline_results_list = []
for (model_name, return_period), result_df in baseline_results.items():
    result_df = result_df.copy()
    result_df['Model'] = model_name
    baseline_results_list.append(result_df)

baseline_results_df = pd.concat(baseline_results_list, ignore_index=True)

# Reorder columns to put Model and Return Period first
cols = ['Model', 'Return Period', 'accuracy', 'auc', 'ci', 'se']
baseline_results_df = baseline_results_df[cols]

print("Baseline Model Results - 1-Day Returns")
print("=" * 50)
baseline_results_df


Random Model 1-day returns- Accuracy: 0.4647, AUC: 0.4751 ± 0.0328, CI: (0.410089224433768, 0.5372236738418149)
Baseline Model Results


,accuracy,auc,ci,se
0,0.464744,0.475084,"(0.410089224433768, 0.5372236738418149)",0.032757


### BERT-based Models Evaluation



#### 1-Day Return Prediction Results:

In [3]:
# Create dataframe with the test_auc, test_auc_ci, test_se, and test_loss for both models under 1-day retirn with the AttenPoolTwoTower and the AttnMLPPoolClassifier architectures

model_bert, test_loss_bert, test_auc_bert, test_auc_ci_bert, test_se_bert = call_model(
    Model="AttnMLPPoolClassifier",
    dim=768,
    attn_hidden=256,
    hidden=256,
    dropout=0.2,
    return_period=1
)

model_fin, test_loss_fin, test_auc_fin, test_auc_ci_fin, test_se_fin = call_model_fin(
    Model="AttnPoolTwoTower",
    dim=768,
    fin_dim=4,
    hidden=256,
    dropout=0.2,
    return_period=1
)

# Create comparison dataframe
results_df = pd.DataFrame({
    'Model': ['AttnMLPPoolClassifier', 'AttnPoolTwoTower'],
    'Test AUC': [test_auc_bert, test_auc_fin],
    'Test AUC CI': [test_auc_ci_bert, test_auc_ci_fin],
    'Test SE': [test_se_bert, test_se_fin],
    'Test Loss': [test_loss_bert, test_loss_fin]
})

epoch 01 | train_loss=0.7057 | val_loss=0.7175 | val_auc=0.467
epoch 02 | train_loss=0.6877 | val_loss=0.7077 | val_auc=0.465
epoch 03 | train_loss=0.7516 | val_loss=0.7172 | val_auc=0.460
epoch 04 | train_loss=0.6985 | val_loss=0.7071 | val_auc=0.479
epoch 05 | train_loss=0.7392 | val_loss=0.7113 | val_auc=0.485
epoch 06 | train_loss=0.7647 | val_loss=0.7121 | val_auc=0.480
epoch 07 | train_loss=0.6416 | val_loss=0.7302 | val_auc=0.474
epoch 08 | train_loss=0.6420 | val_loss=0.7091 | val_auc=0.485
epoch 09 | train_loss=0.6263 | val_loss=0.7137 | val_auc=0.474
epoch 10 | train_loss=0.6845 | val_loss=0.7457 | val_auc=0.472
epoch 11 | train_loss=0.8073 | val_loss=0.7480 | val_auc=0.486
epoch 12 | train_loss=0.6779 | val_loss=0.7518 | val_auc=0.476
epoch 13 | train_loss=0.7063 | val_loss=0.7158 | val_auc=0.495
epoch 14 | train_loss=0.7242 | val_loss=0.7511 | val_auc=0.487
epoch 15 | train_loss=0.6767 | val_loss=0.7252 | val_auc=0.482
epoch 16 | train_loss=0.7223 | val_loss=0.7574 | val_au

In [6]:
# Using the MeanPoolClassifier for 1- day return 

model_bert_mean, test_loss_bert_mean, test_auc_bert_mean, test_auc_ci_bert_mean, test_se_bert_mean = call_model(
    Model="MeanPoolClassifier",
    dim=768,
    attn_hidden=256,
    hidden=256,
    dropout=0.2,
    return_period=1
)

epoch 01 | train_loss=0.6902 | val_loss=0.7105 | val_auc=0.467
epoch 02 | train_loss=0.6591 | val_loss=0.7020 | val_auc=0.442
epoch 03 | train_loss=0.7936 | val_loss=0.7138 | val_auc=0.448
epoch 04 | train_loss=0.6616 | val_loss=0.7033 | val_auc=0.440
epoch 05 | train_loss=0.6635 | val_loss=0.7001 | val_auc=0.442
epoch 06 | train_loss=0.7310 | val_loss=0.7040 | val_auc=0.439
epoch 07 | train_loss=0.6620 | val_loss=0.7055 | val_auc=0.440
epoch 08 | train_loss=0.6906 | val_loss=0.6979 | val_auc=0.444
Early stopping on AUC.


In [4]:
# Get results for MeanPoolClassifier
results_df = meanpooling_withsvm(
    return_period=1
)

results_fin_df = meanpooling_withsvm_fin(
    return_period=1
)

In [7]:
# Combine all results into a single dataframe for 1-day returns
combined_results = pd.DataFrame({
    'Model': ['AttnMLPPoolClassifier (Transcript Only)', 
              'AttnPoolTwoTower (Transcript + Finance)', 
              'MeanPoolClassifier (Transcript Only)', 
              'SVM (Transcript Only)', 
              'SVM (Transcript + Finance)'],
    'Test AUC': [
        test_auc_bert,
        test_auc_fin,
        test_auc_bert_mean,
        results_df['test_auc'],
        results_fin_df['test_auc']
    ],
    'Test AUC CI': [
        test_auc_ci_bert,
        test_auc_ci_fin,
        test_auc_ci_bert_mean,
        results_df['test_auc_ci'],
        results_fin_df['test_auc_ci']
    ],
    'Test SE': [
        test_se_bert,
        test_se_fin,
        test_se_bert_mean,
        results_df['test_se'],
        results_fin_df['test_se']
    ]
})

print("Results for 1-Day Return Prediction")
print("=" * 50)
combined_results

Results for 1-Day Return Prediction


,Model,Test AUC,Test AUC CI,Test SE
0,AttnMLPPoolClassifier (Transcript Only),0.524279,"(0.4574448444567525, 0.5881554715518078)",0.033645
1,AttnPoolTwoTower (Transcript + Finance),0.466000,"(0.4012237762237762, 0.5337906739593956)",0.034077
2,MeanPoolClassifier (Transcript Only),0.483892,"(0.41713535808023994, 0.5501176470588236)",0.035121
3,SVM (Transcript Only),0.422795,"(0.3504740017236426, 0.49157225752970435)",0.035015
4,SVM (Transcript + Finance),0.419930,"(0.36079787738083957, 0.48111173603299584)",0.032169


#### 5-Day Return Prediction Results:

In [8]:
# Create dataframe with the test_auc, test_auc_ci, test_se, and test_loss for both models under 5-day retirn with the AttenPoolTwoTower and the AttnMLPPoolClassifier architectures

model_bert_5d, test_loss_bert_5d, test_auc_bert_5d, test_auc_ci_bert_5d, test_se_bert_5d = call_model(
    Model="AttnMLPPoolClassifier",
    dim=768,
    attn_hidden=256,
    hidden=256,
    dropout=0.2,
    return_period=5
)

model_fin_5d, test_loss_fin_5d, test_auc_fin_5d, test_auc_ci_fin_5d, test_se_fin_5d = call_model_fin(
    Model="AttnPoolTwoTower",
    dim=768,
    fin_dim=4,
    hidden=256,
    dropout=0.2,
    return_period=5
)

# Create comparison dataframe
results_df_5d = pd.DataFrame({
    'Model': ['AttnMLPPoolClassifier', 'AttnPoolTwoTower'],
    'Test AUC': [test_auc_bert_5d, test_auc_fin_5d],
    'Test AUC CI': [test_auc_ci_bert_5d, test_auc_ci_fin_5d],
    'Test SE': [test_se_bert_5d, test_se_fin_5d],
    'Test Loss': [test_loss_bert_5d, test_loss_fin_5d]
})

epoch 01 | train_loss=0.8373 | val_loss=0.7275 | val_auc=0.502
epoch 02 | train_loss=0.7125 | val_loss=0.7004 | val_auc=0.493
epoch 03 | train_loss=0.6626 | val_loss=0.6943 | val_auc=0.499
epoch 04 | train_loss=0.6409 | val_loss=0.7246 | val_auc=0.505
epoch 05 | train_loss=0.6980 | val_loss=0.7064 | val_auc=0.506
epoch 06 | train_loss=0.7819 | val_loss=0.7116 | val_auc=0.505
epoch 07 | train_loss=0.6482 | val_loss=0.7074 | val_auc=0.510
epoch 08 | train_loss=0.6725 | val_loss=0.7681 | val_auc=0.500
epoch 09 | train_loss=0.6344 | val_loss=0.7043 | val_auc=0.511
epoch 10 | train_loss=0.7929 | val_loss=0.7228 | val_auc=0.517
epoch 11 | train_loss=0.7142 | val_loss=0.7336 | val_auc=0.505
epoch 12 | train_loss=0.7843 | val_loss=0.7344 | val_auc=0.493
epoch 13 | train_loss=0.8137 | val_loss=0.7673 | val_auc=0.502
epoch 14 | train_loss=0.4943 | val_loss=0.7304 | val_auc=0.506
epoch 15 | train_loss=0.5313 | val_loss=0.7359 | val_auc=0.501
epoch 16 | train_loss=0.5665 | val_loss=0.7476 | val_au

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# Using the MeanPoolClassifier for 5- day return 

model_bert_mean_5d, test_loss_bert_mean_5d, test_auc_bert_mean_5d, test_auc_ci_bert_mean_5d, test_se_bert_mean_5d = call_model(
    Model="MeanPoolClassifier",
    dim=768,
    attn_hidden=256,
    hidden=256,
    dropout=0.2,
    return_period=5
)

In [ ]:
# Get results for MeanPoolClassifier
results_df_5d = meanpooling_withsvm(
    return_period=5
)

results_fin_df_5d = meanpooling_withsvm_fin(
    return_period=5
)

In [ ]:
# Combine all results into a single dataframe for 5-day returns
combined_results_5d = pd.DataFrame({
    'Model': ['AttnMLPPoolClassifier (Transcript Only)', 
              'AttnPoolTwoTower (Transcript + Finance)', 
              'MeanPoolClassifier (Transcript Only)', 
              'SVM (Transcript Only)', 
              'SVM (Transcript + Finance)'],
    'Test AUC': [
        test_auc_bert,
        test_auc_fin,
        test_auc_bert_mean,
        results_df_5d['test_auc'],
        results_fin_df_5d['test_auc']
    ],
    'Test AUC CI': [
        test_auc_ci_bert,
        test_auc_ci_fin,
        test_auc_ci_bert_mean,
        results_df_5d['test_auc_ci'],
        results_fin_df_5d['test_auc_ci']
    ],
    'Test SE': [
        test_se_bert_5d,
        test_se_fin_5d,
        test_se_bert_mean_5d,
        results_df_5d['test_se'],
        results_fin_df_5d['test_se']
    ]
})

print("Results for 5-Day Return Prediction")
print("=" * 50)
combined_results_5d